In [14]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime

In [15]:
url = 'https://github.com/LeDataSciFi/data/raw/main/Firm%20Year%20Datasets%20(Compustat)/firms2020.csv'
firms_df = pd.read_csv(url).drop_duplicates('tic')

bod_df = pd.read_csv('./Input_data/DirectorComp.csv')
ceo_df = pd.read_csv('./Input_data/CEOComp.csv')

In [17]:
# ceo_df.dropna(axis=1, how='all', inplace=True)
# Calculate the total compensation for the CEO in that year
ceo_df = ceo_df[ceo_df['CEOANN'].notnull()]
ceo_df['TDC1'].describe() #only difference between TDC1 and TDC2 is the options preferences. TDC1 is bk value of options (Black- Scholes equation)

count      4811.000000
mean      11467.801272
std       10203.831261
min           0.001000
25%        6158.732500
50%        9658.115000
75%       14350.184000
max      280621.551000
Name: TDC1, dtype: float64

In [19]:
ceo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4811 entries, 0 to 26762
Data columns (total 89 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EXEC_FULLNAME            4811 non-null   object 
 1   CFOANN                   38 non-null     object 
 2   EXECDIR                  4811 non-null   int64  
 3   OLD_DATAFMT_FLAG         4811 non-null   int64  
 4   INTERLOCK                4811 non-null   int64  
 5   REPRICE                  4811 non-null   int64  
 6   EXECRANK                 64 non-null     float64
 7   CO_PER_ROL               4811 non-null   int64  
 8   CONAME                   4811 non-null   object 
 9   TITLEANN                 4741 non-null   object 
 10  CEOANN                   4811 non-null   object 
 11  SALARY                   4811 non-null   float64
 12  BONUS                    4811 non-null   float64
 13  STOCK_AWARDS             4811 non-null   float64
 14  OPTION_AWARDS          

In [ ]:
# GVKEY: Compustat's Global Company Key
# EXECID: Executive ID number
# YEAR: The year for which you want to calculate the CEO's compensation
# SALARY: Base salary paid to the CEO
# BONUS: Bonus paid to the CEO
# STOCK_AWARDS: Value of stock and option awards paid to the CEO
# OPTIONS_AWARDS: Value of option awards paid to the CEO
# LTIP: Value of long-term incentive plan awards paid to the CEO
# ALL_OTHER_COMP: All other compensation paid to the CEO